In [35]:
# import IPython
# from IPython.display import IFrame
# from IPython.display import display, HTML
# import plotly
# import matplotlib
# %matplotlib inline

### Neo4j
from neo4j import GraphDatabase
from neo4j import  Driver
### Langchain
from langchain_neo4j import Neo4jGraph
from langchain_neo4j import Neo4jVector
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, CSVLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
### ChatModels (https://python.langchain.com/docs/integrations/chat/)
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
### Embeddings
from langchain_ollama import OllamaEmbeddings
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_experimental.graph_transformers import LLMGraphTransformer

from yfiles_jupyter_graphs import GraphWidget
from pydantic import BaseModel, Field

import sys
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from IPython.display import display, HTML
from py2neo import Graph,Node,Relationship
from pyvis.network import Network
import networkx as nx
import random

In [27]:
# import networkx as nx
# import random
# from pyvis.network import Network
# from IPython.display import display, HTML

def visualize_neo4j_graph(graph, query=None, output_file="neo4j_graph_disp.html", show_in_notebook=True):    
    if query is None:
        query = """
        MATCH (n)-[r]->(m)
        WHERE type(r) IN ['LIKES', 'MAKES'] OR true
        RETURN n, r, m LIMIT 100
        """
    
    # Execute query and convert to NetworkX graph
    result = graph.run(query)
    G = nx.DiGraph()
    
    # Define color schemes for different relationship types
    rel_colors = {
        "LIKES": "#FF5733",  # Orange-red
        "MAKES": "#33A1FF",  # Blue
        "DEFAULT": "#CCCCCC"  # Gray for other relationships
    }
    
    # Define node color groups
    node_colors = {
        "Person": "#FFD700",      # Gold
        "Drink": "#98FB98",       # Pale green
        "Manufacturer": "#87CEFA" # Light sky blue
    }
    
    # Add nodes and edges with labels
    for record in result:
        source_node = record["n"]
        target_node = record["m"]
        relationship = record["r"]
        
        source_id = source_node.identity
        target_id = target_node.identity
        
        # Get node labels
        source_label = source_node.get("name", str(source_id))
        target_label = target_node.get("name", str(target_id))
        
        # Get node types
        source_type = list(source_node.labels)[0] if source_node.labels else "Unknown"
        target_type = list(target_node.labels)[0] if target_node.labels else "Unknown"
        
        # Add nodes with attributes including color
        if not G.has_node(source_id):
            G.add_node(source_id, 
                      label=source_label, 
                      title=f"{source_type}: {source_label}", 
                      group=source_type,
                      color=node_colors.get(source_type, "#"+"%06x" % random.randint(0, 0xFFFFFF)))
        if not G.has_node(target_id):
            G.add_node(target_id, 
                      label=target_label, 
                      title=f"{target_type}: {target_label}", 
                      group=target_type,
                      color=node_colors.get(target_type, "#"+"%06x" % random.randint(0, 0xFFFFFF)))
        
        # Add edge with relationship type and appropriate color
        rel_type = type(relationship).__name__
        G.add_edge(source_id, target_id, 
                  title=rel_type,
                  color=rel_colors.get(rel_type, rel_colors["DEFAULT"]),
                  label=rel_type,
                  arrows="to")
    
    # Create PyVis network
    net = Network(notebook=True, height="500px", width="100%")
    # net = Network(notebook=True, height="500px", width="100%", cdn_resources="in_line")

    net.from_nx(G)

    net.set_options("""
    const options = {
      "physics": {
        "barnesHut": {
          "gravitationalConstant": -2000,
          "centralGravity": 0.3,
          "springLength": 150,
          "springConstant": 0.04
        },
        "maxVelocity": 50,
        "minVelocity": 0.75,
        "solver": "barnesHut",
        "timestep": 0.5,
        "stabilization": {
          "enabled": true,
          "iterations": 1000
        }
      },
      "interaction": {
        "hover": true,
        "navigationButtons": true
      },
      "edges": {
        "smooth": {
          "type": "continuous",
          "forceDirection": "none"
        },
        "color": {
          "inherit": false
        },
        "width": 2,
        "font": {
          "size": 12,
          "align": "middle"
        },
        "arrows": {
          "to": {
            "enabled": true,
            "scaleFactor": 0.5
          }
        }
      },
      "nodes": {
        "font": {
          "size": 18,
          "face": "Tahoma"
        },
        "borderWidth": 2,
        "shadow": true
      }
    }
    """)
    
    # Generate and save visualization
    net.show(output_file)
    
    # Display in Jupyter Notebook
    if show_in_notebook:
        display(HTML(output_file))
    
    print(f"Graph visualization saved to {output_file}")


## Py2neo Graph

- [Inspiration](https://github.com/nicolewhite/neo4j-jupyter/tree/master)
- https://nicolewhite.github.io/neo4j-jupyter/hello-world.html

In [53]:
graph = Graph(uri=os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))
graph

Graph('bolt://localhost:7687')

In [54]:
graph.delete_all()

In [55]:
nicole = Node("Person", name="Nicole", age=24)
drew = Node("Person", name="Drew", age=20)

mtdew = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke = Node("Manufacturer", name="Coca Cola")
pepsi = Node("Manufacturer", name="Pepsi")

graph.create(nicole | drew | mtdew | cokezero | coke | pepsi)

In [56]:
graph

Graph('bolt://localhost:7687')

In [57]:
# from py2neo import Relationship
graph.create(Relationship(nicole, "LIKES", cokezero))
graph.create(Relationship(nicole, "LIKES", mtdew))
graph.create(Relationship(drew, "LIKES", mtdew))
graph.create(Relationship(coke, "MAKES", cokezero))
graph.create(Relationship(pepsi, "MAKES", mtdew))

In [58]:
visualize_neo4j_graph(graph)

neo4j_graph_disp.html


Graph visualization saved to neo4j_graph_disp.html


## Neo4j Graph

In [61]:
graph = Neo4jGraph()
graph

In [62]:
def test_connection():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))
    
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) as count").single()
        return f"Connected to Neo4j. Found {result['count']} nodes."

test_connection()

'Connected to Neo4j. Found 6 nodes.'

In [96]:
from yfiles_jupyter_graphs import GraphWidget
def displayGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (n)-[r]->(m) RETURN n, r, m ").graph())
    widget.node_label_mapping = 'id'
    return widget

# # MATCH (n)-[r]->(m) RETURN n, r, m LIMIT 100

# "MATCH (s)-[r:MENTIONS]->(t) RETURN s,r,t"
displayGraph()

GraphWidget(layout=Layout(height='500px', width='100%'))

## Data Loader

In [37]:
loader = CSVLoader(file_path="samples/customers100.csv")
documents = loader.load()

In [38]:
for document in documents:
    print(document)

page_content='Index: 1
Customer Id: DD37Cf93aecA6Dc
First Name: Sheryl
Last Name: Baxter
Company: Rasmussen Group
City: East Leonard
Country: Chile
Phone 1: 229.077.5154
Phone 2: 397.884.0519x718
Email: zunigavanessa@smith.info
Subscription Date: 2020-08-24
Website: http://www.stephenson.com/' metadata={'source': 'samples/customers100.csv', 'row': 0}
page_content='Index: 2
Customer Id: 1Ef7b82A4CAAD10
First Name: Preston
Last Name: Lozano
Company: Vega-Gentry
City: East Jimmychester
Country: Djibouti
Phone 1: 5153435776
Phone 2: 686-620-1820x944
Email: vmata@colon.com
Subscription Date: 2021-04-23
Website: http://www.hobbs.com/' metadata={'source': 'samples/customers100.csv', 'row': 1}
page_content='Index: 3
Customer Id: 6F94879bDAfE5a6
First Name: Roy
Last Name: Berry
Company: Murillo-Perry
City: Isabelborough
Country: Antigua and Barbuda
Phone 1: +1-539-402-0259
Phone 2: (496)978-3969x58947
Email: beckycarr@hogan.com
Subscription Date: 2020-03-25
Website: http://www.lawrence.com/' me

In [39]:
len(documents)

100

## Chunking

- https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/

In [84]:
text_splitter = RecursiveCharacterTextSplitter(separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],chunk_size=250, chunk_overlap=0)
documents_splited = text_splitter.split_documents(documents=documents)
documents_splited

[Document(metadata={'source': 'samples/customers100.csv', 'row': 0}, page_content='Index: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24'),
 Document(metadata={'source': 'samples/customers100.csv', 'row': 0}, page_content='Website: http://www.stephenson.com/'),
 Document(metadata={'source': 'samples/customers100.csv', 'row': 1}, page_content='Index: 2\nCustomer Id: 1Ef7b82A4CAAD10\nFirst Name: Preston\nLast Name: Lozano\nCompany: Vega-Gentry\nCity: East Jimmychester\nCountry: Djibouti\nPhone 1: 5153435776\nPhone 2: 686-620-1820x944\nEmail: vmata@colon.com\nSubscription Date: 2021-04-23'),
 Document(metadata={'source': 'samples/customers100.csv', 'row': 1}, page_content='Website: http://www.hobbs.com/'),
 Document(metadata={'source': 'samples/customers100.csv', 'row': 2}, page_content='Inde

In [85]:
for document in documents_splited:
    print(document)

page_content='Index: 1
Customer Id: DD37Cf93aecA6Dc
First Name: Sheryl
Last Name: Baxter
Company: Rasmussen Group
City: East Leonard
Country: Chile
Phone 1: 229.077.5154
Phone 2: 397.884.0519x718
Email: zunigavanessa@smith.info
Subscription Date: 2020-08-24' metadata={'source': 'samples/customers100.csv', 'row': 0}
page_content='Website: http://www.stephenson.com/' metadata={'source': 'samples/customers100.csv', 'row': 0}
page_content='Index: 2
Customer Id: 1Ef7b82A4CAAD10
First Name: Preston
Last Name: Lozano
Company: Vega-Gentry
City: East Jimmychester
Country: Djibouti
Phone 1: 5153435776
Phone 2: 686-620-1820x944
Email: vmata@colon.com
Subscription Date: 2021-04-23' metadata={'source': 'samples/customers100.csv', 'row': 1}
page_content='Website: http://www.hobbs.com/' metadata={'source': 'samples/customers100.csv', 'row': 1}
page_content='Index: 3
Customer Id: 6F94879bDAfE5a6
First Name: Roy
Last Name: Berry
Company: Murillo-Perry
City: Isabelborough
Country: Antigua and Barbuda
Ph

In [86]:
len(documents_splited)

199

In [87]:
documents_splited[77].page_content

'Website: http://www.farley.org/'

In [88]:
documents_splited[77]

Document(metadata={'source': 'samples/customers100.csv', 'row': 38}, page_content='Website: http://www.farley.org/')

In [89]:
documents_splited[:10]

[Document(metadata={'source': 'samples/customers100.csv', 'row': 0}, page_content='Index: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24'),
 Document(metadata={'source': 'samples/customers100.csv', 'row': 0}, page_content='Website: http://www.stephenson.com/'),
 Document(metadata={'source': 'samples/customers100.csv', 'row': 1}, page_content='Index: 2\nCustomer Id: 1Ef7b82A4CAAD10\nFirst Name: Preston\nLast Name: Lozano\nCompany: Vega-Gentry\nCity: East Jimmychester\nCountry: Djibouti\nPhone 1: 5153435776\nPhone 2: 686-620-1820x944\nEmail: vmata@colon.com\nSubscription Date: 2021-04-23'),
 Document(metadata={'source': 'samples/customers100.csv', 'row': 1}, page_content='Website: http://www.hobbs.com/'),
 Document(metadata={'source': 'samples/customers100.csv', 'row': 2}, page_content='Inde

## Enter LLMs

In [157]:
llm_type = os.getenv("LLM_TYPE", "ollama")
llm_type 

'ollama'

In [158]:
if llm_type == "ollama":
    llm = ChatOllama(model="llama3.2", temperature=0)
    print("Ollama LLM is available.")
elif llm_type == "openai":
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    print("OpenAI.")
else:
    llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)
    print("Claude Shannon here")

llm_transformer = LLMGraphTransformer(llm=llm)

Ollama LLM is available.


In [159]:
# graph_documents = llm_transformer.convert_to_graph_documents(documents[0:10])

In [160]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', properties={}), Node(id='Pierre Curie', properties={}), Node(id='Nobel Prize', properties={})]
Relationships:[]


In [161]:
graph_documents

[GraphDocument(nodes=[Node(id='Marie Curie', properties={}), Node(id='Pierre Curie', properties={}), Node(id='Nobel Prize', properties={})], relationships=[], source=Document(metadata={}, page_content='\nMarie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.\nShe was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.\nHer husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.\nShe was, in 1906, the first woman to become a professor at the University of Paris.\n'))]

In [162]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [163]:
test_connection()

'Connected to Neo4j. Found 4 nodes.'

In [164]:
displayGraph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [165]:
## To delete the grapg
graph= Neo4jGraph()
graph.query("MATCH (n) DETACH DELETE n")

[]

In [166]:
test_connection()

'Connected to Neo4j. Found 0 nodes.'

In [167]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    documents
)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

Nodes:[]
Relationships:[]


In [168]:
graph.add_graph_documents(
    graph_documents_filtered,
    baseEntityLabel=True,
    include_source=True
)

In [169]:
displayGraph()

GraphWidget(layout=Layout(height='500px', width='100%'))

## Back to our csv data

In [187]:
for document in documents_splited:
    print(document)

page_content='Index: 1
Customer Id: DD37Cf93aecA6Dc
First Name: Sheryl
Last Name: Baxter
Company: Rasmussen Group
City: East Leonard
Country: Chile
Phone 1: 229.077.5154
Phone 2: 397.884.0519x718
Email: zunigavanessa@smith.info
Subscription Date: 2020-08-24' metadata={'source': 'samples/customers100.csv', 'row': 0}
page_content='Website: http://www.stephenson.com/' metadata={'source': 'samples/customers100.csv', 'row': 0}
page_content='Index: 2
Customer Id: 1Ef7b82A4CAAD10
First Name: Preston
Last Name: Lozano
Company: Vega-Gentry
City: East Jimmychester
Country: Djibouti
Phone 1: 5153435776
Phone 2: 686-620-1820x944
Email: vmata@colon.com
Subscription Date: 2021-04-23' metadata={'source': 'samples/customers100.csv', 'row': 1}
page_content='Website: http://www.hobbs.com/' metadata={'source': 'samples/customers100.csv', 'row': 1}
page_content='Index: 3
Customer Id: 6F94879bDAfE5a6
First Name: Roy
Last Name: Berry
Company: Murillo-Perry
City: Isabelborough
Country: Antigua and Barbuda
Ph

In [188]:
documents_splited[77].page_content

'Website: http://www.farley.org/'

In [189]:
documents = [Document(page_content=documents_splited[77].page_content)]
documents

[Document(metadata={}, page_content='Website: http://www.farley.org/')]

In [193]:
documents = [Document(page_content=documents_splited.page_content)]
documents

AttributeError: 'list' object has no attribute 'page_content'

In [ ]:
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [196]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=0)
# documents_splited = text_splitter.split_documents(documents=documents)

# Extract page content from each document
documents = [Document(page_content=doc.page_content) for doc in documents_splited]

graph_documents = llm_transformer.convert_to_graph_documents(documents)

print(f"Nodes: {graph_documents[0].nodes}")
print(f"Relationships: {graph_documents[0].relationships}")


Nodes: []
Relationships: []


In [197]:
test_connection()

'Connected to Neo4j. Found 1 nodes.'

In [ ]:
## To delete the grapg
graph= Neo4jGraph()
graph.query("MATCH (n) DETACH DELETE n")

In [ ]:
test_connection()

In [198]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [199]:
displayGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

## Embeddings
- https://medium.com/timescale/finding-the-best-open-source-embedding-model-for-rag-929d1656d331

In [200]:
EMBEDDING_MODELS = [
    {'name':'mxbai-embed-large', 'dimensions': 1024},
    {'name':'nomic-embed-text','dimensions': 768},
    {'name':'bge-m3','dimensions': 1024},
]
 
for model in EMBEDDING_MODELS:
    print(model['name'], model['dimensions'])

mxbai-embed-large 1024
nomic-embed-text 768
bge-m3 1024


In [201]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [202]:
vector_retriever

VectorStoreRetriever(tags=['Neo4jVector', 'OllamaEmbeddings'], vectorstore=<langchain_neo4j.vectorstores.neo4j_vector.Neo4jVector object at 0x000001717DF77A90>, search_kwargs={})

In [203]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext is indexed successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [204]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [205]:
entity_chain.invoke("Who is Sheryl Baxter?")

Entities(names=['Sheryl Baxter'])

## Generate Q&A

In [206]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [207]:
print(graph_retriever("Who is Sheryl Baxter?"))

In [208]:
graph_retriever("Who is Sheryl Baxter?")

''

In [209]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

# Q&A

In [210]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [211]:
chain.invoke(input="Who is Sheryl Baxter?")

'Sheryl Baxter is a customer of Rasmussen Group, based in East Leonard, Chile. She has two phone numbers and an email address associated with her account.'

In [212]:
chain.invoke(input="Tell me more about Sheryl Baxter?")

'Sheryl Baxter is a customer with the ID DD37Cf93aecA6Dc. She has two phone numbers, 229.077.5154 and 397.884.0519x718, and an email address zunigavanessa@smith.info. Her subscription date was August 24, 2020.'

In [214]:
chain.invoke(input="Who company is Bruce Esparza from? and is he has an email ?")

'Bruce Esparza is from Huerta-Mclean. Yes, he has an email address: preese@frye-vega.com.'